In [9]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

# Simulation Parameters
L = 10.0   # Length of the domain
T = 1.0    # Total simulation time
Nx = 100   # Number of spatial grid points
Nt = 201   # Number of time steps
dx = L / (Nx - 1)  # Spatial step size
dt = T / Nt        # Time step size

# Monte Carlo Simulations
for o in range(1000):
    D = np.random.uniform(1, 100)  # Diffusion coefficient
    R = abs(np.random.uniform(D - 10, D + 10)) % 100  # Sorption coefficient
    alpha = (D / R) * dt / dx**2  # Stability parameter

    print(f"Iteration {o}: D={D:.2f}, R={R:.2f}, alpha={alpha:.4f}")

    if alpha > 0.5:
        print("Skipping: alpha > 0.5 (unstable)")
        continue  # Skip this iteration if unstable

    # Initialize concentration array
    C = np.zeros(Nx)  
    C[int(Nx / 4):int(Nx / 2)] = 1.0  # Initial condition: step function

    # Boundary conditions (Dirichlet: fixed concentration)
    C[0] = 0.0
    C[-1] = 0.0

    # Store concentration data over time
    C_all = np.zeros((Nt, Nx))  

    # Time-stepping loop
    for t in range(Nt):
        C_new = C.copy()
        for i in range(1, Nx - 1):
            C_new[i] = C[i] + alpha * (C[i + 1] - 2 * C[i] + C[i - 1])
        C = C_new

        # Apply boundary conditions at each time step
        C[0] = 0.0
        C[-1] = 0.0

        # Store current concentration profile
        C_all[t, :] = C

    # Save results to HDF5 file
    try:
        with h5py.File("diff_sorp.h5", "a") as f:
            group_name = f"{o}"
            while group_name in f:
                group_name += "_retry"
            g = f.create_group(group_name)
            g.create_dataset("D", data=D)
            g.create_dataset("R", data=R)
            g.create_dataset("data", data=C_all)
    except Exception as e:
        print(f"Error saving iteration {o}: {e}")

print("Simulation completed.")


Iteration 0: D=37.98, R=29.26, alpha=0.6330
Skipping: alpha > 0.5 (unstable)
Iteration 1: D=40.17, R=40.08, alpha=0.4887
Iteration 2: D=30.43, R=32.91, alpha=0.4509
Iteration 3: D=45.71, R=51.23, alpha=0.4351
Iteration 4: D=77.03, R=79.72, alpha=0.4712
Iteration 5: D=29.51, R=21.74, alpha=0.6619
Skipping: alpha > 0.5 (unstable)
Iteration 6: D=44.67, R=46.16, alpha=0.4720
Iteration 7: D=80.66, R=80.66, alpha=0.4876
Iteration 8: D=31.10, R=36.02, alpha=0.4210
Iteration 9: D=3.14, R=13.01, alpha=0.1175
Iteration 10: D=7.52, R=16.92, alpha=0.2168
Iteration 11: D=38.36, R=44.32, alpha=0.4221
Iteration 12: D=21.74, R=25.68, alpha=0.4129
Iteration 13: D=70.66, R=61.92, alpha=0.5565
Skipping: alpha > 0.5 (unstable)
Iteration 14: D=99.14, R=95.39, alpha=0.5068
Skipping: alpha > 0.5 (unstable)
Iteration 15: D=76.62, R=74.64, alpha=0.5005
Skipping: alpha > 0.5 (unstable)
Iteration 16: D=75.00, R=68.93, alpha=0.5305
Skipping: alpha > 0.5 (unstable)
Iteration 17: D=79.37, R=79.46, alpha=0.4871
Iter

In [10]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from tqdm import tqdm
import h5py


def visualize_burgers(xcrd, data, path):
    """
    This function animates the Burgers equation

    Args:
    path : path to the desired file
    param: PDE parameter of the data shard to be visualized
    """
    fig, ax = plt.subplots()
    ims = []

    for i in tqdm(range(data.shape[0])):
        if i == 0:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        else:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        ims.append([im[0]])

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

    writer = animation.PillowWriter(fps=15, bitrate=1800)
    ani.save(path, writer=writer)
    plt.close(fig)

visualize_burgers([i for i in range(100)], C_all, "a.gif")

100%|██████████| 201/201 [00:00<00:00, 5545.25it/s]


In [11]:
from scipy.stats import skewnorm

with h5py.File("diff_sorp.h5", "a") as f:
    for key in f.keys():
        u = f[key]["data"][:]
        noise = skewnorm.rvs(a=1, scale=0.2, size=u.shape)
        u_noise = u + noise
        f[key].create_dataset("noise", data=u_noise)

In [13]:
import h5py

with h5py.File("/home/pes1ug22am100/Documents/Research and Experimentation/NoisyICML/pde-gen/diff-sorp/diff_sorp.h5", "r") as f:
    print("Keys in file:", list(f.keys()))
    if "10" in f:
        print("Keys in '10':", list(f["10"].keys()))
    else:
        print("'10' not found in the file.")


Keys in file: ['1', '10', '100', '101', '103', '106', '108', '11', '111', '112', '115', '116', '118', '12', '121', '122', '123', '127', '129', '130', '133', '135', '137', '138', '139', '140', '142', '143', '144', '146', '148', '149', '150', '152', '153', '154', '157', '159', '160', '163', '165', '166', '167', '169', '17', '172', '173', '176', '178', '179', '180', '184', '185', '188', '190', '193', '196', '199', '2', '201', '202', '203', '204', '206', '209', '213', '215', '216', '218', '219', '223', '229', '230', '231', '232', '233', '234', '236', '237', '238', '239', '240', '241', '242', '243', '244', '249', '25', '252', '253', '256', '257', '258', '259', '264', '266', '267', '269', '27', '270', '271', '272', '276', '277', '279', '280', '282', '283', '284', '287', '288', '291', '292', '295', '296', '298', '299', '3', '30', '300', '304', '305', '306', '307', '308', '310', '312', '314', '316', '317', '318', '319', '321', '322', '323', '324', '326', '327', '328', '329', '330', '332', '333